In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"


In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

import utils
from tqdm import tqdm

teacher = torchvision.models.vit_b_16(pretrained = True)
student = torchvision.models.vit_b_16(pretrained = False)
# model = torchvision.models.vit_l_32(pretrained=True)

print(f"patch_size : {student.patch_size}")
print(f"image_size : {student.image_size}")
print(f"hidden_dim : {student.hidden_dim}")

student = utils.VisionTransformer(student)
teacher = utils.VisionTransformer(teacher)


patch_size : 16
image_size : 224
hidden_dim : 768


In [ ]:
torchvision.datasets.

In [3]:
normalize = transforms.Normalize(mean=[0.5074,0.4867,0.4411],
                                 std=[0.2011,0.1987,0.2025])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
])

dataset_val = torchvision.datasets.ImageNet(root= "../ImagenetData/data", split='val', transform = transform)
dataset_train = torchvision.datasets.ImageNet(root= "../ImagenetData/data", split='train', transform = transform_train)

val_loader = torch.utils.data.DataLoader(dataset_val,
                                          batch_size=128,
                                          shuffle=True,
                                          num_workers=4)

train_loader = torch.utils.data.DataLoader(dataset_train,
                                          batch_size=128,
                                          shuffle=True,
                                          num_workers=4)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
S_optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)
T_optimizer = optim.SGD(teacher.parameters(), lr=0.001, momentum=0.9)
CE_loss = nn.CrossEntropyLoss()

S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1, 2, 3,4], gamma=0.1)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[40,80], gamma=0.1)

student = student.cuda()
teacher = teacher.cuda()



In [ ]:
import numpy as np

criterion_onlylabel = lambda a,b : mse(a*b, b)

criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))

    
import torch
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)
past_acc = 0.0
stack = 0  
for epoch in range(100):
    print(f"lr : {S_scheduler.get_last_lr()}")
    T_correct = 0
    S_correct = 0
    all_data = 0
    
    loss_distill = []
    loss_CE = []
    loss_response = []
    student = student.train()
    for img, label in tqdm(train_loader):
        input_data = img.cuda()
        label = label.cuda()
        
        
        all_data += len(input_data)
        input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=0.9, std = 0.4).cuda()
        
        S_optimizer.zero_grad()
        T_optimizer.zero_grad()

        layer = random.randint(0,  2)
        input_data, fk = student(input_data,layer)
        input_lrp, fk_lrp = teacher(input_lrp,layer)
        
        distill_loss = mse(fk, fk_lrp)
                    
        CE_loss = criterion_CE(input_data, label)
        
#         response_loss = criterion_response(input_data, input_lrp)
        
        T_correct += sum(label == torch.argmax(input_lrp, dim=1))
        S_correct += sum(label == torch.argmax(input_data, dim=1))
        
        loss_CE.append(CE_loss.item())
        loss_distill.append(distill_loss.item())

        loss = (distill_loss * 2 + CE_loss) / 2
        loss.backward()
        S_optimizer.step()
    print("distill loss : ", sum(loss_distill) / len(loss_distill))
    print("general loss : ", sum(loss_CE) / len(loss_CE))
#     print("response loss : ", sum(loss_response) / len(loss_response))
    
    print(f"Teacher acc: {T_correct / all_data}")
    print(f"Student acc: {S_correct / all_data}")
    test_acc = utils.test(student, val_loader, epoch) # student도 변하는거 확인 완료함
    utils.test(teacher, val_loader, epoch) # student도 변하는거 확인 완료함
    
    if test_acc > past_acc + 0.01:
        stack = 0
    elif past_acc + 0.005 > test_acc:
        stack+=1
    past_acc = test_acc
    
    
    if stack > 2:
        S_scheduler.step()
        stack = 0
        
    print("=" * 100)

lr : [0.01]


100%|██████████| 10010/10010 [3:26:10<00:00,  1.24s/it] 


distill loss :  0.15275222984227269
general loss :  5.479379260218465
Teacher acc: 0.9393154978752136
Student acc: 0.06781473010778427


100%|██████████| 391/391 [01:16<00:00,  5.10it/s]


0 	 test acc : 0.11017999798059464


...

100%|██████████| 391/391 [01:16<00:00,  5.14it/s]


25 	 test acc : 0.5523599982261658


100%|██████████| 391/391 [01:16<00:00,  5.11it/s]


25 	 test acc : 0.810699999332428
lr : [1e-05]


 19%|█▉        | 1894/10010 [38:57<2:45:55,  1.23s/it]

In [ ]:
def test(model, data, epoch = 0):
    all_data = 0
    correct = 0
    model = model.eval()
    for img, label in tqdm(data):
        model.eval()
        with torch.no_grad():
            img = img.cuda()
            label = label.cuda()

            output, _ = model(img)

            correct += sum(label == torch.argmax(output, dim=1))
            all_data += len(img)
    print(f"{epoch} \t test acc : {correct / all_data}")
    return correct / all_data

In [ ]:
student = torchvision.models.vit_b_16(pretrained = True).cuda()
student = utils.VisionTransformer(student)
test(student, val_loader, 0)

In [ ]:
torchvision.models.vit_b_16(pretrained = True).encoder.layers